# Creating and Evaluating Solutions 

To recap from the first notebook:

For the most part the algorithms in Amazon Personalize look to solve different tasks explained here:

1. HRNN & HRNN-Metadata - Personalization
1. HRNN Coldstart - Personalization that promotes new conten
1. Personalized-Ranking - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. SIMS(Similar Items) - Given one item, what other items are also interacted with by users.
1. Popularity-Count - What items are most popular, if HRNN or HRNN-Metadata do not have an answer for the user you query, this is what is returned by default.


No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. UserID - User who interacted
1. ItemID - Item the user interacted with
1. Timestamp - When did this interaction occur

We also support event types and event values defined by:

1. Event Type - Categorical label of an event (browse, purcahsed, rated, etc).
1. Event Value - Something corresponding to event type that happened. Generally speaking we look to normalized between 0 and 1 for the values over the types. So if there are three phases to complete a transaction (clicked, added-to-cart, and purchased) there would be an event_value for each phase as 0.33, 0.66, 1.0 respectfully.

In this particular exercise we will leave event_type and event_value ignored. They can come in handy later but are skipped for the initial POC. 

The previous notebooks covered:

1. Selecting a dataset.
1. Preparing interactions data for Personalize.
1. Preparing item or user metadata for Personalize [Optional].
1. Creating a Dataset Group.
1. Creating and importing data into an Interactions dataset within the dataset group.
1. Creating and importing data into the metadata datasets [Optional].


## Creating Solutions

This nobeook will cover creating the following solutions:

1. HRNN
1. SIMS
1. Personalized-Ranking

After that the metrics will be explained and another notebook will showcase how to interact with the Solutions once they are deployed into a Campaign.

The first step is to reload the imports and the stored variables from the previous notebooks.

In [1]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from datetime import datetime

In [2]:
%store -r

In [3]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.


In [4]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 65000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2019, 6, 20, 0, 39, 17, 64000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   'stat

That is just a JSON representation of all of the algorithms that we have mentioned already. 

Next we will select a particular algorithm then build a model with it.

### HRNN


HRNN is one of the more advanced recommendation models that you can use and it allows for things like real-time updates of recommendations based on user behavior. It also tends to out perform other approaches like collaborative filtering. We will kick this job off first as it takes the longest to complete.

#### Select Recipe

In [5]:
HRNN_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

#### Create and Wait for Solution
First you will create the solution with the API, then you will create a version. 

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [6]:
hrnn_create_solution_response = personalize.create_solution(
    name = "personalize-poc-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = HRNN_recipe_arn
)

hrnn_solution_arn = hrnn_create_solution_response['solutionArn']
print(json.dumps(hrnn_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-hrnn",
  "ResponseMetadata": {
    "RequestId": "15dd1d2d-851b-478f-8b29-dbd53f03dba0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:47 GMT",
      "x-amzn-requestid": "15dd1d2d-851b-478f-8b29-dbd53f03dba0",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [7]:
hrnn_create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

In [8]:
hrnn_solution_version_arn = hrnn_create_solution_version_response['solutionVersionArn']
print(json.dumps(hrnn_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-hrnn/ac6547d9",
  "ResponseMetadata": {
    "RequestId": "c91df915-15c7-4b3c-ae75-3bf2d4a704ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:48 GMT",
      "x-amzn-requestid": "c91df915-15c7-4b3c-ae75-3bf2d4a704ab",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS


SIMS is one of the longest running algorithms within Amazon for recommendation systems. A core use case for it is when you have one item and you want to recommend items that have been intereacted with in similar ways over your entire user base(not personalized). Sometimes this leads to recommending mostly popular items, so there is a hyperparameter that can be tweaked that will reduce the popular items in your results. 

Just as last time we start by selecting the recipie:

#### Select Recipe

In [9]:
## SIMS:
SIMS_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

#### Create and Wait for Solution

As with HRNN, start with the solution first.

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [10]:
sims_create_solution_response = personalize.create_solution(
    name = "personalize-poc-sims",
    datasetGroupArn = dataset_group_arn,
    recipeArn = SIMS_recipe_arn
)

sims_solution_arn = sims_create_solution_response['solutionArn']
print(json.dumps(sims_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-sims",
  "ResponseMetadata": {
    "RequestId": "a907d9f2-5979-49cb-84f4-d9e6e419e322",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:53 GMT",
      "x-amzn-requestid": "a907d9f2-5979-49cb-84f4-d9e6e419e322",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [11]:
sims_create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)

In [12]:
sims_solution_version_arn = sims_create_solution_version_response['solutionVersionArn']
print(json.dumps(sims_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-sims/c1c7c204",
  "ResponseMetadata": {
    "RequestId": "0c2fee95-1dfc-412a-94cf-37d8987f6a07",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:53 GMT",
      "x-amzn-requestid": "0c2fee95-1dfc-412a-94cf-37d8987f6a07",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Personalized Ranking is an interesting application of HRNN. Instead of just recommending what is most probable for your user in question, this algorithm takes in a user and a list of items as well. The items are then rendered back in the order of most probability for the user. The use case here is for filtering on genre for example, or when you have a broad collection that you would like better ordered for a particular user.

#### Select Recipe

In [13]:
#### Personalized-rerank
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Create and Wait for Solution
First you will create the solution with the API, then you will create a version. 

Note the solution is just a label kind of identifier, you'll also need to create a version which is the actual trained model.

In [14]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-poc-rerank",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "fc79ab3d-2286-4012-8a2c-b5f160aa08f7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:56 GMT",
      "x-amzn-requestid": "fc79ab3d-2286-4012-8a2c-b5f160aa08f7",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the Solution Version

This process will actually take a while to complete, upwards of 25 minutes on. Normally there would be while loops to poll until the task is completed. However the task would block other cells from executing and the goal here is to create many models and deploy them quickly. Below there are instructions to viewing the progress in browser. After creating all of the solution versions go there and watch for updates.

In [15]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [16]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-rerank/9cfa4956",
  "ResponseMetadata": {
    "RequestId": "64b60b7c-3945-4d79-b2ef-127203193dc1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:03:58 GMT",
      "x-amzn-requestid": "64b60b7c-3945-4d79-b2ef-127203193dc1",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Viewing Solution Creation Status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Solutions and recipes`.
* You will now see a list of all of the solutions you created above. Click any one of them. 
* Note in `Solution versions` the job that is in progress. Once it is `Active` you solution is ready to be reviewed. It is also capable of being deployed.


## Evaluating Solutions

After about an hour max the solutions should be ready for review. While they are in progress it is a good idea to cover the various algorithms and their behavior in depth. You'll have another lull period as the solutions are being deployed into campaigns as well, so you can split the material into 2 sections if that makes it easier. Also it can be a good time to discuss alternatives to how the data was fed into the system and what kind of results to expect from it.

The firt step is to obtain the solutions metrics, API calls for each below.

### HRNN Metrics:

In [17]:
hrnn_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(hrnn_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-hrnn/ac6547d9",
  "metrics": {
    "coverage": 0.0426,
    "mean_reciprocal_rank_at_25": 0.0424,
    "normalized_discounted_cumulative_gain_at_10": 0.062,
    "normalized_discounted_cumulative_gain_at_25": 0.0787,
    "normalized_discounted_cumulative_gain_at_5": 0.048,
    "precision_at_10": 0.0105,
    "precision_at_25": 0.0071,
    "precision_at_5": 0.0133
  },
  "ResponseMetadata": {
    "RequestId": "446a9bc3-739c-40ed-b0b4-3bf4f91ba5ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:57:46 GMT",
      "x-amzn-requestid": "446a9bc3-739c-40ed-b0b4-3bf4f91ba5ab",
      "content-length": "402",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


The above metrics tell us that at 5 items, we have less than a 1% chance (.8% literally) in a recommendation being a part of a user's interaction history (in the hold out phase from training and validation). This is clearly not a great model, but keep in mind instead of plays, all we had to go on is that they tagged a particular artist and that is it.

Something else to note, the users' history is influenced by ANY recommendation model that is in place as your historical data is being collected. This often means that while your model probably won't be this bad with a customer or on your own data, it does bias the metrics to favor their existing solution. If you work to just push the offline metrics to match or exceed their existing solution you may just be making HRNN start to behave like whatever they were already using.

This is a great time to have a conversation about AB testing and to think about the actual business outcomes they are looking to drive. From there you look to run small experiments with Personalize against their existing recommendation system and see over time how the AB test performs. If Personalize is winning then it is the time to filter more and more traffic to Personalize and campaigns within it. Over time the bias from the existing model will fade.

### SIMS Metrics:

In [18]:
sims_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(sims_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-sims/c1c7c204",
  "metrics": {
    "coverage": 0.0841,
    "mean_reciprocal_rank_at_25": 0.0334,
    "normalized_discounted_cumulative_gain_at_10": 0.0483,
    "normalized_discounted_cumulative_gain_at_25": 0.0551,
    "normalized_discounted_cumulative_gain_at_5": 0.0347,
    "precision_at_10": 0.0087,
    "precision_at_25": 0.0051,
    "precision_at_5": 0.0069
  },
  "ResponseMetadata": {
    "RequestId": "4a707fb5-06d7-40fa-acbe-63673567fc45",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:57:48 GMT",
      "x-amzn-requestid": "4a707fb5-06d7-40fa-acbe-63673567fc45",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In this example we are seeing a slightly elivated precision at 5, a little over 1% at 1.04% this time. Effectively this is probably within the margin of error but given that no effort was made to mask popularity, it may just be returning super popular results that a large volume of users have interacted with in some way. 

### Personalized Ranking Metrics:

In [19]:
rerank_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = rerank_solution_version_arn
)

print(json.dumps(rerank_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:059124553121:solution/personalize-poc-rerank/9cfa4956",
  "metrics": {
    "coverage": 0.0021,
    "mean_reciprocal_rank_at_25": 0.0325,
    "normalized_discounted_cumulative_gain_at_10": 0.0501,
    "normalized_discounted_cumulative_gain_at_25": 0.0596,
    "normalized_discounted_cumulative_gain_at_5": 0.0428,
    "precision_at_10": 0.0091,
    "precision_at_25": 0.0057,
    "precision_at_5": 0.0126
  },
  "ResponseMetadata": {
    "RequestId": "56f4093e-100d-4ccf-a535-2793b3e52ceb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jan 2020 18:57:51 GMT",
      "x-amzn-requestid": "56f4093e-100d-4ccf-a535-2793b3e52ceb",
      "content-length": "406",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Storing Useful Variables]

Before exiting this notebook, run the following cells to save off our version arns for use later.

In [20]:
%store hrnn_solution_version_arn
%store sims_solution_version_arn
%store rerank_solution_version_arn

Stored 'hrnn_solution_version_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'rerank_solution_version_arn' (str)


Just a quick comment on this one, here we see again a precision of near 1%, as this is based on HRNN, that is to be expected. 

You are now ready to move on to deploying and interacting with campaigns, continue by opening `Deploying_Campaigns_and_Interacting.ipynb`.